In [48]:
#import
import cv2
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
from torchvision import models
from PIL import Image
import pandas as pd

import spatial_transforms as ST
from models.ResNet import AP3DResNet50

In [49]:
## 4 Frames 
img1 = Image.open("000003.jpg")
img2 = Image.open("000004.jpg")
img3 = Image.open("000005.jpg")
img4 = Image.open("000006.jpg")

In [50]:
open("gt.txt", "r")
Koordinaten = open("gt.txt", "r")

data = pd.read_csv('gt.txt', header = None)
data.columns = ['Frame', 'ID', 'x', 'y', 'w', 'h', 'Dies', 'Das', 'Visability']

data.drop(data[data.ID != 2].index, inplace=True)
data = data.loc[data['Frame'].isin(['3','4','5','6'])]
data.drop(['Frame', 'ID', 'Dies', 'Das', 'Visability'], axis=1, inplace=True)
print(data)

x1 = data.at[1052, 'x']
y1 = data.at[1052, 'y']
w1 = data.at[1052, 'w']
h1 = data.at[1052, 'h']

x2 = data.at[1053, 'x']
y2 = data.at[1053, 'y']
w2 = data.at[1053, 'w']
h2 = data.at[1053, 'h']

x3 = data.at[1054, 'x']
y3 = data.at[1054, 'y']
w3 = data.at[1054, 'w']
h3 = data.at[1054, 'h']

x4 = data.at[1055, 'x']
y4 = data.at[1055, 'y']
w4 = data.at[1055, 'w']
h4 = data.at[1055, 'h']

        x    y   w    h
1052  372  407  80  239
1053  372  406  81  239
1054  373  405  81  239
1055  373  404  82  238


In [51]:
## hardcode 4 bounding boxes
bbox1 = img1.crop((x1, y1, x1+w1, y1+h1))
bbox2 = img2.crop((x2, y2, x2+w2, y2+h2))
bbox3 = img3.crop((x3, y3, x3+w3, y3+h3))
bbox4 = img4.crop((x4, y4, x4+w4, y4+h4))

bbox1.save('bbox1.png')
bbox2.save('bbox2.png')
bbox3.save('bbox3.png')
bbox4.save('bbox4.png')

In [52]:
##brauche ich für das 3DCNN wieder die normalization und resize???
##transform = transforms.Compose([
##    transforms.Resize(256),
##    transforms.ToTensor(),
##    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
##])

##transform = ST.Compose([
##    ST.Scale((256, 128), interpolation=3),
##    ST.ToTensor(),
##    ST.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
##])

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 128), interpolation=3),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

bbox1_trans = transform (bbox1)
bbox2_trans = transform (bbox2)
bbox3_trans = transform (bbox3)
bbox4_trans = transform (bbox4)

print(bbox1_trans.size())
print(bbox2_trans.size())
print(bbox3_trans.size())
print(bbox4_trans.size())

torch.Size([3, 256, 128])
torch.Size([3, 256, 128])
torch.Size([3, 256, 128])
torch.Size([3, 256, 128])


In [64]:
### Soll man nur 3 Frames nehmen für das CNN? 

###bboxtensor = torch.stack([bbox1_trans, bbox2_trans, bbox3_trans, bbox4_trans])
bboxtensor = torch.stack([bbox1_trans, bbox2_trans, bbox3_trans])
bboxtensor.size()
bboxtensor_5dim = bboxtensor.unsqueeze(0)

bboxtensor_5dim.size()

torch.Size([1, 3, 3, 256, 128])

In [67]:
##def AP3DResNet50(num_classes, **kwargs):
##    c3d_idx = [[],[0, 2],[0, 2, 4],[]]
##    nl_idx = [[],[],[],[]]
##    return ResNet503D(num_classes, AP3D.APP3DC, c3d_idx, nl_idx, **kwargs)


CNN3D = AP3DResNet50(1261)
CNN3D.load_state_dict(torch.load('/home/ga27qef/thesis/resnet3d_mars.pth.tar'))

RuntimeError: Error(s) in loading state_dict for ResNet503D:
	Missing key(s) in state_dict: "conv1.weight", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "layer1.0.conv1.weight", "layer1.0.bn1.weight", "layer1.0.bn1.bias", "layer1.0.bn1.running_mean", "layer1.0.bn1.running_var", "layer1.0.conv2.weight", "layer1.0.bn2.weight", "layer1.0.bn2.bias", "layer1.0.bn2.running_mean", "layer1.0.bn2.running_var", "layer1.0.conv3.weight", "layer1.0.bn3.weight", "layer1.0.bn3.bias", "layer1.0.bn3.running_mean", "layer1.0.bn3.running_var", "layer1.0.downsample.0.weight", "layer1.0.downsample.1.weight", "layer1.0.downsample.1.bias", "layer1.0.downsample.1.running_mean", "layer1.0.downsample.1.running_var", "layer1.1.conv1.weight", "layer1.1.bn1.weight", "layer1.1.bn1.bias", "layer1.1.bn1.running_mean", "layer1.1.bn1.running_var", "layer1.1.conv2.weight", "layer1.1.bn2.weight", "layer1.1.bn2.bias", "layer1.1.bn2.running_mean", "layer1.1.bn2.running_var", "layer1.1.conv3.weight", "la

In [66]:
features = CNN3D(bboxtensor_5dim)

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 2048])